In [ ]:
# metodo simplex
EPS = 1e-12  # tolerancia numerica

# utilidades de lectura
def leer_int(msg, min_val=None, max_val=None):
    # lee un entero con validacion opcional
    valor = None
    listo = False
    while not listo:
        txt = input(msg).strip()
        ok = True
        try:
            valor_tmp = int(txt)
        except Exception:
            ok = False
        if ok:
            if min_val is not None:
                if valor_tmp < min_val:
                    ok = False
            if max_val is not None and ok:
                if valor_tmp > max_val:
                    ok = False
        if ok:
            valor = valor_tmp
            listo = True
        else:
            print("Entrada inválida. Intenta de nuevo.")
    return valor

def leer_float(msg):
    # lee un real con reintentos
    valor = None
    listo = False
    while not listo:
        txt = input(msg).strip()
        ok = True
        try:
            valor_tmp = float(txt)
        except Exception:
            ok = False
        if ok:
            valor = valor_tmp
            listo = True
        else:
            print("Entrada inválida. Intenta de nuevo.")
    return valor

# construccion del tableau
def construir_tabla(c, A, b):
    # arma: [ indicador_z | x1..xn | s1..sm | rhs ]
    n = len(c)
    m = len(A)
    col_RHS = n + m + 1
    ncols = col_RHS + 1

    filaZ = [0.0] * ncols
    i = 0
    while i < ncols:
        filaZ[i] = 0.0
        i += 1
    filaZ[0] = 1.0
    j = 0
    while j < n:
        filaZ[1 + j] = -float(c[j])
        j += 1

    tabla = [filaZ]
    i = 0
    while i < m:
        fila = [0.0] * ncols
        j = 0
        while j < n:
            fila[1 + j] = float(A[i][j])
            j += 1
        fila[1 + n + i] = 1.0
        fila[col_RHS] = float(b[i])
        tabla.append(fila)
        i += 1

    return tabla, n, m, col_RHS

# seleccion de columna entrante
def elegir_columna_entrante(tabla, n, m, col_RHS):
    # elige la mas negativa en fila z (excluye rhs)
    filaZ = tabla[0]
    j = 1
    col_elegida = -1
    mas_neg = 0.0
    while j < col_RHS:
        val = filaZ[j]
        if val < mas_neg - EPS or (abs(val - mas_neg) <= EPS and val < mas_neg):
            mas_neg = val
            col_elegida = j
        j += 1
    return col_elegida

# seleccion de fila saliente
def elegir_fila_saliente(tabla, col, m, col_RHS):
    # usa razon minima rhs/a_ij con a_ij>0, desempate por menor indice
    i = 1
    fila_elegida = -1
    mejor_ratio = None
    while i <= m:
        aij = tabla[i][col]
        if aij > EPS:
            rhs = tabla[i][col_RHS]
            ratio = rhs / aij
            if mejor_ratio is None:
                mejor_ratio = ratio
                fila_elegida = i
            else:
                if ratio < mejor_ratio - EPS or (abs(ratio - mejor_ratio) <= EPS and i < fila_elegida):
                    mejor_ratio = ratio
                    fila_elegida = i
        i += 1
    return fila_elegida

# pivoteo
def pivotear(tabla, fila_pivote, col_pivote, col_RHS):
    # normaliza fila pivote y anula columna en el resto
    pivote = tabla[fila_pivote][col_pivote]
    j = 0
    ncols = col_RHS + 1
    while j < ncols:
        tabla[fila_pivote][j] = tabla[fila_pivote][j] / pivote
        j += 1

    i = 0
    nfilas = len(tabla)
    while i < nfilas:
        if i != fila_pivote:
            factor = tabla[i][col_pivote]
            if abs(factor) > EPS:
                j = 0
                while j < ncols:
                    tabla[i][j] = tabla[i][j] - factor * tabla[fila_pivote][j]
                    j += 1
        i += 1

# impresion de tabla
def _fmt_num(x, prec):
    s = ("{0:." + str(prec) + "f}").format(x)
    return s

def imprimir_tabla(tabla, prec=4):
    # imprime filas z, f01..fm
    print("Tabla actual:")
    i = 0
    while i < len(tabla):
        if i == 0:
            encabezado = "Z : "
        else:
            if i < 10:
                encabezado = "F0" + str(i) + ": "
            else:
                encabezado = "F" + str(i) + ": "
        fila_txt = encabezado
        j = 0
        while j < len(tabla[i]):
            fila_txt += _fmt_num(tabla[i][j], prec)
            if j < len(tabla[i]) - 1:
                fila_txt += "  "
            j += 1
        print(fila_txt)
        i += 1
    print("")

# reconstruccion de solucion
def _es_col_canonica(tabla, col, fila_obj, EPS):
    # verifica columna tipo e_i (1 en una fila, 0 en el resto y 0 en fila z)
    i = 0
    nfilas = len(tabla)
    uno_en = -1
    ok = True
    while i < nfilas:
        val = tabla[i][col]
        if i == fila_obj:
            if abs(val) > EPS:
                ok = False
        else:
            if abs(val - 1.0) <= EPS and uno_en == -1:
                uno_en = i
            elif abs(val) > EPS:
                ok = False
        i += 1
    return ok and (uno_en != -1), uno_en

def reconstruir_solucion(tabla, n, m, col_RHS):
    # lee x, slacks y z desde el tableau final
    x = [0.0] * n
    j = 0
    while j < n:
        es_canon, fila = _es_col_canonica(tabla, 1 + j, 0, EPS)
        if es_canon and fila >= 1:
            x[j] = tabla[fila][col_RHS]
        else:
            x[j] = 0.0
        j += 1

    sl = [0.0] * m
    j = 0
    while j < m:
        es_canon, fila = _es_col_canonica(tabla, 1 + n + j, 0, EPS)
        if es_canon and fila >= 1:
            sl[j] = tabla[fila][col_RHS]
        else:
            sl[j] = 0.0
        j += 1

    Z = tabla[0][col_RHS]
    return Z, x, sl

# simplex
def simplex(c, A, b, mostrar_pasos=True, prec=4, max_iter=10000):
    # ciclo: entrante -> saliente -> pivoteo hasta optimo/no_acotado
    tabla, n, m, col_RHS = construir_tabla(c, A, b)
    iteracion = 0
    estado = "EN_PROCESO"
    continuar = True

    while continuar:
        col = elegir_columna_entrante(tabla, n, m, col_RHS)
        if col == -1:
            estado = "OPTIMO"
            continuar = False
        else:
            fila = elegir_fila_saliente(tabla, col, m, col_RHS)
            if fila == -1:
                estado = "NO_ACOTADO"
                continuar = False
            else:
                if mostrar_pasos:
                    print("--> Pivote: fila {}, columna {} (valor {})".format(
                        fila, col, _fmt_num(tabla[fila][col], prec)))
                pivotear(tabla, fila, col, col_RHS)
                if mostrar_pasos:
                    imprimir_tabla(tabla, prec=prec)
        iteracion = iteracion + 1
        if iteracion >= max_iter and continuar:
            continuar = False

    Z, x, sl = reconstruir_solucion(tabla, n, m, col_RHS)
    return {"estado": estado, "Z": Z, "x": x, "slacks": sl, "tabla": tabla}

# entrada de modelo
def input_modelo():
    # pide n, m, c, a, b para ax<=b
    print("=== Modelo: Max c^T x  s.a.  Ax <= b, x >= 0, b >= 0 ===")
    n = leer_int("Número de variables (n): ", 1, None)
    m = leer_int("Número de restricciones (m): ", 1, None)

    c = []
    idx = 0
    while idx < n:
        val = leer_float("c{}: ".format(idx + 1))
        c.append(val)
        idx += 1

    A = []
    b = []
    i = 0
    while i < m:
        print("Restricción R{}:".format(i + 1))
        fila = []
        j = 0
        while j < n:
            val = leer_float("  a{}{}: ".format(i + 1, j + 1))
            fila.append(val)
            j += 1
        A.append(fila)
        bi = leer_float("  b{} : ".format(i + 1))
        b.append(bi)
        i += 1

    return c, A, b

# demo
def demo_kyndryl():
    # max 60x1 + 50x2; 3x1+2x2<=240; 2x1+1x2<=200; 1x1+2x2<=100
    c = [60.0, 50.0]
    A = [
        [3.0, 2.0],
        [2.0, 1.0],
        [1.0, 2.0]
    ]
    b = [240.0, 200.0, 100.0]
    return c, A, b

# metodo grafico con matplotlib
def _det(a1, b1, a2, b2):
    return a1*b2 - a2*b1

def _interseccion(l1, l2):
    # l=(a,b,c) representa a*x + b*y = c
    a1 = l1[0]; b1 = l1[1]; c1 = l1[2]
    a2 = l2[0]; b2 = l2[1]; c2 = l2[2]
    d = _det(a1, b1, a2, b2)
    if abs(d) < EPS:
        return None
    x = (c1*b2 - c2*b1) / d
    y = (a1*c2 - a2*c1) / d
    return (x, y)

def _factible_xy(x, y, A, b):
    # verifica x>=0,y>=0 y a1*x+a2*y<=b
    if x < -EPS or y < -EPS:
        return False
    i = 0
    while i < len(A):
        a1 = A[i][0]; a2 = A[i][1]
        if a1*x + a2*y > b[i] + EPS:
            return False
        i += 1
    return True

def grafico_matplotlib_2vars(c, A, b, titulo="Region factible y optimo (n=2)"):
    # requiere matplotlib solo para graficar
    import matplotlib.pyplot as plt

    # construye lineas Ax=b y ejes
    lineas = []
    i = 0
    while i < len(A):
        lineas.append((A[i][0], A[i][1], b[i]))
        i += 1
    lineas.append((1.0, 0.0, 0.0))  # x=0
    lineas.append((0.0, 1.0, 0.0))  # y=0

    # intersecciones factibles
    pts = []
    i = 0
    while i < len(lineas):
        j = i + 1
        while j < len(lineas):
            P = _interseccion(lineas[i], lineas[j])
            if P is not None:
                x = P[0]; y = P[1]
                if _factible_xy(x, y, A, b):
                    if x < 0.0 and x > -EPS:
                        x = 0.0
                    if y < 0.0 and y > -EPS:
                        y = 0.0
                    pts.append((x, y))
            j += 1
        i += 1

    # agrega origen y cortes con ejes si aplican
    pts.append((0.0, 0.0))
    k = 0
    while k < len(A):
        a1 = A[k][0]; a2 = A[k][1]
        if a2 > EPS:
            y = b[k] / a2
            if y >= -EPS and _factible_xy(0.0, y, A, b):
                pts.append((0.0, y))
        if a1 > EPS:
            x = b[k] / a1
            if x >= -EPS and _factible_xy(x, 0.0, A, b):
                pts.append((x, 0.0))
        k += 1

    # elimina duplicados aproximados
    uniq = []
    p = 0
    while p < len(pts):
        x = pts[p][0]; y = pts[p][1]
        existe = False
        q = 0
        while q < len(uniq):
            X = uniq[q][0]; Y = uniq[q][1]
            if abs(X - x) < 1e-9 and abs(Y - y) < 1e-9:
                existe = True
            q += 1
        if not existe:
            uniq.append((x, y))
        p += 1
    pts = uniq

    if len(pts) == 0:
        print("No hay región factible.")
        return

    # elige mejor vertice por c^T x
    bx = 0.0; by = 0.0; bz = None
    r = 0
    while r < len(pts):
        x = pts[r][0]; y = pts[r][1]
        z = c[0]*x + c[1]*y
        if bz is None:
            bz = z; bx = x; by = y
        else:
            if z > bz + EPS:
                bz = z; bx = x; by = y
        r += 1

    # calcula rangos
    xmax = 0.0; ymax = 0.0
    t = 0
    while t < len(pts):
        if pts[t][0] > xmax:
            xmax = pts[t][0]
        if pts[t][1] > ymax:
            ymax = pts[t][1]
        t += 1
    if bx > xmax:
        xmax = bx
    if by > ymax:
        ymax = by
    xmax = xmax * 1.1 + 1.0
    ymax = ymax * 1.1 + 1.0
    if xmax < 5.0:
        xmax = 5.0
    if ymax < 5.0:
        ymax = 5.0

    # poligono de region factible ordenado por angulo
    cx = 0.0; cy = 0.0
    s = 0
    while s < len(pts):
        cx += pts[s][0]; cy += pts[s][1]
        s += 1
    cx = cx / float(len(pts))
    cy = cy / float(len(pts))

    # orden por atan2
    import math
    angs = []
    u = 0
    while u < len(pts):
        angs.append((math.atan2(pts[u][1]-cy, pts[u][0]-cx), pts[u]))
        u += 1
    # simple bubble-like
    angs.sort(key=lambda t: t[0])
    pts_sorted = []
    w = 0
    while w < len(angs):
        pts_sorted.append(angs[w][1])
        w += 1

    xs = []
    ys = []
    a = 0
    while a < len(pts_sorted):
        xs.append(pts_sorted[a][0])
        ys.append(pts_sorted[a][1])
        a += 1
    xs.append(pts_sorted[0][0])
    ys.append(pts_sorted[0][1])

    # traza
    plt.figure()
    plt.fill(xs, ys, alpha=0.2, label="region factible")

    # fronteras de restricciones
    # generamos segmentos simples entre cortes con ejes para ilustrar
    i = 0
    while i < len(A):
        a1 = A[i][0]; a2 = A[i][1]; bi = b[i]
        Xs = []; Ys = []
        # interseccion con x=0 y con y=0 si existen
        if abs(a2) > EPS:
            y0 = bi / a2
            if y0 >= -EPS:
                Xs.append(0.0); Ys.append(y0)
        if abs(a1) > EPS:
            x0 = bi / a1
            if x0 >= -EPS:
                Xs.append(x0); Ys.append(0.0)
        if len(Xs) == 2:
            plt.plot(Xs, Ys, linewidth=1)
        i += 1

    # ejes
    plt.plot([0, xmax],[0, 0], linewidth=1)
    plt.plot([0, 0],[0, ymax], linewidth=1)

    # punto optimo
    plt.scatter([bx],[by], s=60, marker='o', label="optimo ({:.2f},{:.2f}) Z={:.2f}".format(bx, by, bz))

    # isoprofit por el optimo
    Xiso = [0.0, xmax]
    Yiso = []
    j = 0
    while j < 2:
        xval = Xiso[j]
        if abs(c[1]) > EPS:
            yval = (bz - c[0]*xval) / c[1]
        else:
            yval = by
        Yiso.append(yval)
        j += 1
    plt.plot(Xiso, Yiso, linestyle='--', label="isoprofit")

    plt.xlim(0, xmax)
    plt.ylim(0, ymax)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title(titulo)
    plt.legend()
    plt.grid(True)
    plt.show()

# menu
def main():
    salir = False
    while not salir:
        print("===============================================")
        print(" Simplex por consola + Grafico (matplotlib, n=2)")
        print(" 1) Ingresar modelo y resolver (Simplex)")
        print(" 2) Demo Kyndryl (Simplex)")
        print(" 3) Metodo Grafico (matplotlib, n=2)")
        print(" 4) Salir")
        print("===============================================")
        op = leer_int("Opción: ", 1, 4)

        if op == 1:
            c, A, b = input_modelo()
            resp_pasos = input("¿Mostrar pasos (s/n)? ").strip().lower()
            mostrar_pasos = (resp_pasos == "s")
            prec = leer_int("Decimales a imprimir (0-10): ", 0, 10)
            res = simplex(c, A, b, mostrar_pasos=mostrar_pasos, prec=prec)
            print("Estado:", res["estado"])
            print("Z*   :", _fmt_num(res["Z"], prec))
            i = 0
            salida_x = "x*   : ["
            while i < len(res["x"]):
                salida_x += _fmt_num(res["x"][i], prec)
                if i < len(res["x"]) - 1:
                    salida_x += ", "
                i += 1
            salida_x += "]"
            print(salida_x)
            i = 0
            salida_s = "Slacks: ["
            while i < len(res["slacks"]):
                salida_s += _fmt_num(res["slacks"][i], prec)
                if i < len(res["slacks"]) - 1:
                    salida_s += ", "
                i += 1
            salida_s += "]"
            print(salida_s)
            print("")
        elif op == 2:
            c, A, b = demo_kyndryl()
            resp_pasos = input("¿Mostrar pasos (s/n)? ").strip().lower()
            mostrar_pasos = (resp_pasos == "s")
            prec = leer_int("Decimales a imprimir (0-10): ", 0, 10)
            res = simplex(c, A, b, mostrar_pasos=mostrar_pasos, prec=prec)
            print("Estado:", res["estado"])
            print("Z*   :", _fmt_num(res["Z"], prec))
            i = 0
            salida_x = "x*   : ["
            while i < len(res["x"]):
                salida_x += _fmt_num(res["x"][i], prec)
                if i < len(res["x"]) - 1:
                    salida_x += ", "
                i += 1
            salida_x += "]"
            print(salida_x)
            i = 0
            salida_s = "Slacks: ["
            while i < len(res["slacks"]):
                salida_s += _fmt_num(res["slacks"][i], prec)
                if i < len(res["slacks"]) - 1:
                    salida_s += ", "
                i += 1
            salida_s += "]"
            print(salida_s)
            print("")
        elif op == 3:
            print("=== método gráfico (matplotlib, n=2) ===")
            c = []
            idx = 0
            while idx < 2:
                val = leer_float("c{}: ".format(idx + 1))
                c.append(val)
                idx += 1
            m = leer_int("Número de restricciones (m): ", 1, None)
            A = []
            b = []
            i = 0
            while i < m:
                print("Restricción R{} (a1*x1 + a2*x2 <= b):".format(i + 1))
                fila = []
                j = 0
                while j < 2:
                    val = leer_float("  a{}{}: ".format(i + 1, j + 1))
                    fila.append(val)
                    j += 1
                A.append(fila)
                bi = leer_float("  b{} : ".format(i + 1))
                b.append(bi)
                i += 1
            grafico_matplotlib_2vars(c, A, b, titulo="Region factible y optimo")
        else:
            salir = True



In [3]:
main()

 Simplex por consola (estilo TORA)
 1) Ingresar modelo y resolver
 2) Demo Kyndryl (Max 60A + 50B)
 3) Salir
Opción: 2
¿Mostrar pasos (s/n)? s
Decimales a imprimir (0-10): 4
--> Pivote: fila 1, columna 1 (valor 3.0000)
Tabla actual:
Z : 1.0000  0.0000  -10.0000  20.0000  0.0000  4800.0000
F01: 0.0000  1.0000  0.6667  0.3333  0.0000  80.0000
F02: 0.0000  0.0000  -0.3333  -0.6667  1.0000  40.0000
F03: 0.0000  0.0000  1.3333  -0.3333  0.0000  20.0000

--> Pivote: fila 3, columna 2 (valor 1.3333)
Tabla actual:
Z : 1.0000  0.0000  0.0000  17.5000  0.0000  4950.0000
F01: 0.0000  1.0000  0.0000  0.5000  0.0000  70.0000
F02: 0.0000  0.0000  0.0000  -0.7500  1.0000  45.0000
F03: 0.0000  0.0000  1.0000  -0.2500  0.0000  15.0000

Estado: OPTIMO
Z*   : 4950.0000
x*   : [70.0000, 15.0000]
Slacks: [0.0000, 45.0000, 0.0000]

 Simplex por consola (estilo TORA)
 1) Ingresar modelo y resolver
 2) Demo Kyndryl (Max 60A + 50B)
 3) Salir
Opción: 3
